This notebook performs the same task as DistanceComputtion.ipynb but for the topics, ie it computes the distance matrix for each votation subjects based on the topic modelling results.

In [2]:
import pandas as pd
import glob
import os
import numpy as np
import matplotlib.pyplot as plt
import sklearn
import sklearn.ensemble
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score, train_test_split, cross_val_predict, learning_curve
import sklearn.metrics

%matplotlib inline
%load_ext autoreload
%autoreload 2

# There's a lot of columns in the DF. 
# Therefore, we add this option so that we can see more columns
pd.options.display.max_columns = 100

In [38]:
path = '../datas/treated_data/Vote/'
voting_no_topics_df = pd.read_csv(path+'legiid_47-50.csv')
voting_no_topics_df = voting_no_topics_df.drop_duplicates(['BillTitle'], keep = 'last')
print(len(voting_no_topics_df))
voting_no_topics_df.head(3)

3415


,Unnamed: 0,VoteEnd,BillTitle,BusinessTitle,Subject,MeaningNo,MeaningYes,BusinessShortNumber,ID,IdLegislativePeriod,IdSession
6,2615,2009-09-07,Arrêté fédéral concernant la contribution de l...,Réduction des disparités économiques et social...,Entrer en matière,Proposition de la minorité Stamm (ne pas entre...,Proposition de la majorité (entrer en matière),9.0550,6398,48,4811
7,2616,2009-09-07,Renforcement du Traité sur la non-prolifératio...,Renforcement du Traité sur la non-prolifératio...,NaN,Rejeter la motion,Adopter la motion,7.3676,6400,48,4811
8,2617,2009-09-07,Une zone exempte d'armes nucléaires au coeur d...,Une zone exempte d'armes nucléaires au coeur d...,NaN,Rejeter la motion,Adopter la motion,7.3802,6401,48,4811


In [22]:
voting_no_topics_df.shape

(8573, 11)

In [4]:
path = '../datas/nlp_results/'
voting_df = pd.read_csv(path+'voting_with_topics.csv')

print('Entries in the DataFrame',voting_df.shape)

#Dropping the useless column
voting_df = voting_df.drop('Unnamed: 0',1)

#Putting numerical values into the columns that should have numerical values
#print(voting_df.columns.values)

num_cols = ['Decision', ' armée', ' asile / immigration', ' assurances', ' budget', ' dunno', ' entreprise/ finance',
           ' environnement', ' famille / enfants', ' imposition', ' politique internationale', ' retraite  ']
voting_df[num_cols] = voting_df[num_cols].apply(pd.to_numeric)

#Inserting the full name at the second position
voting_df.insert(2,'Name', voting_df['FirstName'] + ' ' + voting_df['LastName'])

voting_df.head(5)

Entries in the DataFrame (1713854, 39)


,BillTitle,BusinessNumber,Name,BusinessShortNumber,BusinessTitle,Canton,CantonID,CantonName,Decision,DecisionText,FirstName,ID,IdLegislativePeriod,IdSession,IdVote,Language,LastName,MeaningNo,MeaningYes,ParlGroupCode,ParlGroupColour,ParlGroupName,ParlGroupNameAbbreviation,PersonNumber,RegistrationNumber,Subject,VoteEnd,text,armée,asile / immigration,assurances,budget,dunno,entreprise/ finance,environnement,famille / enfants,imposition,politique internationale,retraite
0,Arrêté fédéral concernant la contribution de l...,20090055,Chiara Simoneschi-Cortesi,9.055,Réduction des disparités économiques et social...,TI,21,Tessin,7,Die Präsidentin/der Präsident stimmt nicht,Chiara,1183799,48,4811,6392,FR,Simoneschi-Cortesi,Proposition de la minorité Stamm (ne pas entre...,Proposition de la majorité (entrer en matière),CEg,#FFFFA500,Groupe conservateur-catholique,Groupe K,511,2652,Entrer en matière,2009-09-07T15:44:26.69,Arrêté fédéral concernant la contribution de l...,0.006995,0.930066,0.006993,0.006993,0.006993,0.006994,0.006993,0.006993,0.006993,0.006993,0.006993
1,Arrêté fédéral concernant la contribution de l...,20090055,Pascale Bruderer Wyss,9.055,Réduction des disparités économiques et social...,AG,1,Aargau,5,Hat nicht teilgenommen,Pascale,1183800,48,4811,6392,FR,Bruderer Wyss,Proposition de la minorité Stamm (ne pas entre...,Proposition de la majorité (entrer en matière),S,#FFFF0000,Groupe socialiste,Groupe S,823,2652,Entrer en matière,2009-09-07T15:44:26.69,Arrêté fédéral concernant la contribution de l...,0.006995,0.930066,0.006993,0.006993,0.006993,0.006994,0.006993,0.006993,0.006993,0.006993,0.006993
2,Arrêté fédéral concernant la contribution de l...,20090055,Guy Parmelin,9.055,Réduction des disparités économiques et social...,VD,23,Waadt,5,Hat nicht teilgenommen,Guy,1183801,48,4811,6392,FR,Parmelin,Proposition de la minorité Stamm (ne pas entre...,Proposition de la majorité (entrer en matière),V,#FF006400,"Groupe des Paysans, Artisans et Bourgeois",Groupe B,1108,2652,Entrer en matière,2009-09-07T15:44:26.69,Arrêté fédéral concernant la contribution de l...,0.006995,0.930066,0.006993,0.006993,0.006993,0.006994,0.006993,0.006993,0.006993,0.006993,0.006993
3,Arrêté fédéral concernant la contribution de l...,20090055,Jean-René Germanier,9.055,Réduction des disparités économiques et social...,VS,24,Wallis,5,Hat nicht teilgenommen,Jean-René,1183802,48,4811,6392,FR,Germanier,Proposition de la minorité Stamm (ne pas entre...,Proposition de la majorité (entrer en matière),RL,#FF00BFFF,Groupe radical-démocratique,Groupe R,1117,2652,Entrer en matière,2009-09-07T15:44:26.69,Arrêté fédéral concernant la contribution de l...,0.006995,0.930066,0.006993,0.006993,0.006993,0.006994,0.006993,0.006993,0.006993,0.006993,0.006993
4,Arrêté fédéral concernant la contribution de l...,20090055,Edi Engelberger,9.055,Réduction des disparités économiques et social...,NW,14,Nidwalden,5,Hat nicht teilgenommen,Edi,1183803,48,4811,6392,FR,Engelberger,Proposition de la minorité Stamm (ne pas entre...,Proposition de la majorité (entrer en matière),RL,#FF00BFFF,Groupe radical-démocratique,Groupe R,318,2652,Entrer en matière,2009-09-07T15:44:26.69,Arrêté fédéral concernant la contribution de l...,0.006995,0.930066,0.006993,0.006993,0.006993,0.006994,0.006993,0.006993,0.006993,0.006993,0.006993


We first erase the duplicates and only collect the results of the topic modelling for each votation

In [81]:
voting_df_copy = voting_df.drop_duplicates(['BillTitle', 'BusinessTitle'], keep = 'last')
print(len(voting_df_copy))
for i in voting_df_copy.index:
    if str(voting_df_copy.loc[i].BillTitle) == 'nan':
            voting_df_copy.set_value(i,'BillTitle',voting_df_copy.loc[i].BusinessTitle)
            
voting_df_copy = voting_df_copy.drop_duplicates(['BillTitle'], keep = 'last')
print(len(voting_df_copy))

3470
3415


In [82]:

voting_subjects = voting_df_copy['BillTitle'].unique()
topics = [' armée', ' asile / immigration', ' assurances', ' budget', ' dunno', ' entreprise/ finance', ' environnement', ' famille / enfants', ' imposition', ' politique internationale', ' retraite  ']
print("{n} subjects voted in the parliament from 2009 to 2015".format(n=voting_subjects.shape[0]))
voting_df_copy = voting_df_copy.set_index(['BillTitle'])
voting_df_copy = voting_df_copy[topics]
voting_df_copy.head()

3415 subjects voted in the parliament from 2009 to 2015


,armée,asile / immigration,assurances,budget,dunno,entreprise/ finance,environnement,famille / enfants,imposition,politique internationale,retraite
BillTitle,,,,,,,,,,,
Arrêté fédéral concernant la contribution de la Suisse en faveur de la Bulgarie et de la Roumanie au titre de la réduction des disparités économiques et sociales dans l'Union européenne élargie,0.006995,0.930066,0.006993,0.006993,0.006993,0.006994,0.006993,0.006993,0.006993,0.006993,0.006993
Renforcement du Traité sur la non-prolifération des armes nucléaires,0.018182,0.018185,0.018183,0.271557,0.564795,0.018183,0.018182,0.018183,0.018183,0.018183,0.018184
Une zone exempte d'armes nucléaires au coeur de l'Europe,0.015152,0.389858,0.015152,0.232520,0.256405,0.015152,0.015152,0.015152,0.015152,0.015154,0.015152
Boycott de la liste des terroristes établie par l'ONU,0.022727,0.772700,0.022731,0.022729,0.022728,0.022728,0.022728,0.022736,0.022731,0.022732,0.022731
Ratification du Protocole de l'ONU sur les armes à feu et mise en oeuvre de l'instrument Thalmann,0.012987,0.549980,0.012987,0.012987,0.333134,0.012987,0.012987,0.012987,0.012987,0.012987,0.012987


We then implement the distance function, which is simply the euclidean distance between the vectors whose entries are the percentage for each topic computed by topic modelling.

In [84]:
def distance(p1, p2):
    return np.linalg.norm(p1-p2)

We then apply it to every pairs of subjects in order to compute the distance matrix.

In [85]:
n = voting_subjects.shape[0]
distanceMatrix = np.zeros((n,n))

for i in range(n):
    if i % 10 == 0:
        print("Compute distances from subject " + str(i))
    for j in range(n):
        distanceMatrix[i][j] = distance(voting_df_copy.loc[voting_subjects[i]].values,
                                        voting_df_copy.loc[voting_subjects[j]].values)
print("Mean distance : {d}".format(d = np.mean(distanceMatrix)))

Compute distances from subject 0
Compute distances from subject 10
Compute distances from subject 20
Compute distances from subject 30
Compute distances from subject 40
Compute distances from subject 50
Compute distances from subject 60
Compute distances from subject 70
Compute distances from subject 80
Compute distances from subject 90
Compute distances from subject 100
Compute distances from subject 110
Compute distances from subject 120
Compute distances from subject 130
Compute distances from subject 140
Compute distances from subject 150
Compute distances from subject 160
Compute distances from subject 170
Compute distances from subject 180
Compute distances from subject 190
Compute distances from subject 200
Compute distances from subject 210
Compute distances from subject 220
Compute distances from subject 230
Compute distances from subject 240
Compute distances from subject 250
Compute distances from subject 260
Compute distances from subject 270
Compute distances from subject 

We save the matrix. We observe as expected that the diagonal of the distance matrix contains only 0 as the distance between some subject and itself is 0.

In [86]:
import pandas as pd 
df = pd.DataFrame(distanceMatrix, index = voting_subjects, columns = voting_subjects)
df.to_csv("distanceMatrixSubjects.csv")
df.head()

,Arrêté fédéral concernant la contribution de la Suisse en faveur de la Bulgarie et de la Roumanie au titre de la réduction des disparités économiques et sociales dans l'Union européenne élargie,Renforcement du Traité sur la non-prolifération des armes nucléaires,Une zone exempte d'armes nucléaires au coeur de l'Europe,Boycott de la liste des terroristes établie par l'ONU,Ratification du Protocole de l'ONU sur les armes à feu et mise en oeuvre de l'instrument Thalmann,Accompagner la construction du nouvel Etat du Kosovo,Renonciation à des projets d'aide au développement menés par l'Etat,Meilleure protection juridique pour les défenseurs de l'environnement,La Suisse doit oeuvrer pour préserver l'unité de la Macédoine,Promouvoir l'Observatoire du Conseil de l'Europe pour le respect des droits de l'homme en Palestine et en Israël,Eclaircissements relatifs au Traité sur la non-prolifération des armes nucléaires,Vérité des coûts au DFAE,Halte au versement des contributions à la Bulgarie,Contribution de la Suisse à la scolarisation d'un million d'enfants africains,Prévention des génocides. Combattre les génocides culturels,"Loi fédérale sur l'examen et le contrôle de la sécurité technique (Loi sur le contrôle de la sécurité, LCS)","Loi fédérale sur les installations de transport par conduites de combustibles ou carburants liquides ou gazeux (Loi sur les installations de transport par conduites, LITC)",Arrêté fédéral concernant le crédit-cadre de la Confédération pour la réalisation de la 1ère étape de la 3e correction du Rhône (R3) pour la période de 2009 à 2014,Interdiction dès 2012 des ampoules inefficaces,Remplacement des ampoules à filament par des ampoules économiques,"Révision partielle de la LTC. Requête de la Comcom, de la Comco et du Surveillant des prix",Mesures visant à réguler la population des oiseaux piscivores et à indemniser les dégâts causés à la pêche professionnelle,Politique énergétique extérieure de la Suisse,Paysage ferroviaire suisse. Consolidation par les CFF,Certificat énergétique cantonal des bâtiments,Promouvoir l'efficacité énergétique dans le secteur du bâtiment en accordant des avantages en termes d'utilisation des sols pour la construction,Extension de la taxe sur le CO2 aux émissions grises,Les redevances de la SSR doivent être baissées de 20 pour cent,Programmes de réduction des émissions de CO2,Faciliter l'engagement politique,Transports publics en comparaison internationale,Lutte contre les risques naturels,Déchets. Des alternatives à l'incinération,Manifestations aériennes en Suisse. Décréter un moratoire pour préserver le climat,CO2 et gouvernement d'entreprise,Loi fédérale sur l'assurance-maladie (LAMal) (Mesures pour endiguer l'évolution des coûts),Opération NAVFOR Atalanta. Service d'appui à l'étranger et révision de la LAAM,"Loi fédérale sur l'encouragement de la culture (Loi sur l'encouragement de la culture, LEC)",Sécurité sociale pour les professions avec changements de place fréquents ou engagements de durée limitée,Loi fédérale sur les allégements fiscaux en faveur des familles avec enfants,Allègements fiscaux en faveur des familles qui gardent elles-mêmes leurs enfants,Passer au système de l'imposition individuelle,"Loi fédérale sur des mesures de stabilisation conjoncturelle temporaire dans les domaines du marché du travail, des technologies de l'information et de la communication et du pouvoir d'achat",Employés des services douaniers à la retraite vivant dans la Principauté du Liechtenstein. Imposition en fonction du domicile,Traité douanier entre la Suisse et le Liechtenstein. Domicile légal des parents des employés des services douaniers,Mesures incitatives pour la construction d'installations photovoltaïques,Pas d'indemnités de départ pour le personnel de la Confédération,Lutte contre la corruption. Interdire le pantouflage,Pas de deniers publics pour les prix décernés par la Confédération,Résoudre les problèmes de financement des projets FTP dans le cadre de la troisième phase des

We finally compute for each subject the topic which appears the most.

In [87]:
topic_df = pd.DataFrame(index = voting_subjects)
topic_df['Topic'] = voting_df_copy[topics].idxmax(axis=1)
topic_df.head()

,Topic
Arrêté fédéral concernant la contribution de la Suisse en faveur de la Bulgarie et de la Roumanie au titre de la réduction des disparités économiques et sociales dans l'Union européenne élargie,asile / immigration
Renforcement du Traité sur la non-prolifération des armes nucléaires,dunno
Une zone exempte d'armes nucléaires au coeur de l'Europe,asile / immigration
Boycott de la liste des terroristes établie par l'ONU,asile / immigration
Ratification du Protocole de l'ONU sur les armes à feu et mise en oeuvre de l'instrument Thalmann,asile / immigration


In [88]:
topic_df.to_csv("SubjectTopicMapping.csv")